In [ ]:
!pip install hazm

In [ ]:
!pip install torch
import torch
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00


In [ ]:
import hazm
from hazm import *
import pprint
from hazm import stopwords_list
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import BertTokenizer, BertModel

model_name = 'HooshvareLab/bert-fa-zwnj-base'  # Specify the BERT model variant
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a folder in the root directory
!mkdir -p "/content/drive/My Drive/XSum Dataset/Mashregh Dataset/ParsBERT Pretrained"

In [ ]:
# test_column = df_1['document']
file_directory = '/content/drive/My Drive/XSum Dataset/Mashregh Dataset/ParsBERT Pretrained/parsbert_persian_Jahan.csv'

In [ ]:
n=-1
df = pd.read_csv(r'Jahan.csv')
test_Column = df['Text']
for x in test_Column:
    n = n + 1
    text = x

    #Tokenizing and Removing Stopwords using hazm
    sentence_tokens = []
    normalizer = hazm.Normalizer()
    sentences=hazm.sent_tokenize(normalizer.normalize(text))
    sentence_tokens_included_stopwords=[hazm.word_tokenize(sentence) for sentence in sentences]
    for sent in sentence_tokens_included_stopwords:
      for w in sent:
        if w in stopwords_list():
          sent.remove(w)
    sentence_tokens = sentence_tokens_included_stopwords
    temp_list = []
    for sent in sentence_tokens:
      for word in sent:
        listToStr = ' '.join(map(str, sent))
      temp_list.append(listToStr)
    new_text = ' '.join(map(str, temp_list))
    new_sentences=hazm.sent_tokenize(normalizer.normalize(new_text))

    #Embedding with Pretrained Persian Word Vectors (FastText)
    array_of_sentence_embeddings = []
    for newsent in new_sentences:
      encoded_input = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          padding='max_length',
          truncation=True,
          max_length=150,  # Specify the desired maximum length of the sequence
          return_tensors='pt'
        )
      input_ids = encoded_input['input_ids']
      attention_mask = encoded_input['attention_mask']
      with torch.no_grad():
          outputs = model(input_ids, attention_mask=attention_mask)
          words_embedding = outputs.last_hidden_state
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      sentence_embedding = torch.mean(words_embedding, dim=1).to(device)  # Shape: [1, 768]
      sentence_embedding = sentence_embedding.squeeze(0)
      array_of_sentence_embeddings.append(sentence_embedding)

    #Calculating Simularity using Cosine Simularity
    similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
    for i,row_embedding in enumerate(array_of_sentence_embeddings):
        for j,column_embedding in enumerate(array_of_sentence_embeddings):
            similarity_matrix[i][j]=abs(1-spatial.distance.cosine(row_embedding,column_embedding))

    #Creating Graph
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
    top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
    print("<<<text " + str(n+1) + " summarized>>>")
    temp_summary_list = []
    for sent in sentences:
        if sent in top.keys():
          temp_summary_list.append(sent)
    summary=str(temp_summary_list)
    summary=summary.replace("[","")
    summary=summary.replace("]","")
    summary=summary.replace(","," ")
    summary=summary.replace("\'","")
    summary=summary.replace("\\u200c"," ")
    df.loc[n, 'Textrank'] = summary
df.to_csv(file_directory)

df.head()


<<<text 1 summarized>>>
<<<text 2 summarized>>>
<<<text 3 summarized>>>
<<<text 4 summarized>>>
<<<text 5 summarized>>>
<<<text 6 summarized>>>
<<<text 7 summarized>>>
<<<text 8 summarized>>>
<<<text 9 summarized>>>
<<<text 10 summarized>>>
<<<text 11 summarized>>>
<<<text 12 summarized>>>
<<<text 13 summarized>>>
<<<text 14 summarized>>>
<<<text 15 summarized>>>
<<<text 16 summarized>>>
<<<text 17 summarized>>>
<<<text 18 summarized>>>
<<<text 19 summarized>>>
<<<text 20 summarized>>>
<<<text 21 summarized>>>
<<<text 22 summarized>>>
<<<text 23 summarized>>>
<<<text 24 summarized>>>
<<<text 25 summarized>>>
<<<text 26 summarized>>>
<<<text 27 summarized>>>
<<<text 28 summarized>>>
<<<text 29 summarized>>>
<<<text 30 summarized>>>
<<<text 31 summarized>>>
<<<text 32 summarized>>>
<<<text 33 summarized>>>
<<<text 34 summarized>>>
<<<text 35 summarized>>>
<<<text 36 summarized>>>
<<<text 37 summarized>>>
<<<text 38 summarized>>>
<<<text 39 summarized>>>
<<<text 40 summarized>>>
<<<text 4